# 爬痞客幫

In [17]:
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL="""https://www.pixnet.net/searcharticle?q=%E6%97%A5%E5%8A%87%E5%A0%B4%E6%99%AF+%E8%87%AA%E7%94%B1%E8%A1%8C&type=related&period=year&page={}"""
# URL="""https://www.pixnet.net/searcharticle?q=%E6%97%A5%E5%8A%87%E5%A0%B4%E6%99%AF+%E8%87%AA%E7%94%B1%E8%A1%8C&type=related&period=year&page=1"""
blog = []




# for page in range(1, 30):
#     res = r.get(URL.format(page))
#     res.encoding = 'utf-8'
#     soup = BeautifulSoup(res.text, 'lxml')
#     articles = soup.select('li.search-list')
#     for article in articles:
#         article_dict = {}
#         article_dict['title'] = article.select_one('li.search-title').select_one('a')['title']
#         article_dict['blogger_name'] = article.select_one('div.search-meta').select_one('a').text
#         article_dict['popularity'] = article.select('span.search-views')[0].select_one('span').text.strip()
#         article_dict['post_time'] = article.select('span.search-postTime')[0].text
#         article_dict['url'] = article.select_one('li.search-title').select_one('a')['href']
#         blog.append(article_dict)
#         print article_dict

for page in range(1, 2):
    res = r.get(URL.format(page))
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    articles = soup.select('li.search-list')
    for x,article in enumerate(articles):
        article_dict = {}
        article_dict['title'] = article.select_one('li.search-title').select_one('a')['title']
        article_dict['blogger_name'] = article.select_one('div.search-meta').select_one('a').text
        article_dict['popularity'] = article.select('span.search-views')[0].select_one('span').text.strip()
        article_dict['post_time'] = article.select('span.search-postTime')[0].text
        PAGE_URL = article.select_one('li.search-title').select_one('a')['href']
        # 以此URL抓取頁面
        driver = webdriver.Remote("http://localhost:9515", webdriver.DesiredCapabilities.CHROME)
        try:
            driver.get(PAGE_URL)
#             with open('blog_page.html', 'w') as f:
#                 f.write(driver.page_source.encode('utf-8'))
            element = WebDriverWait(driver,15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.back-to-top")))
            driver.execute_script("return arguments[0].scrollIntoView()", element)
            time.sleep(3)
            with open('blog_page_{}_{}.html'.format(page, x), 'w') as f:  #{}1--第幾頁ˋ; {}2--第幾篇 
                f.write(driver.page_source.encode('utf-8'))
        finally:
            driver.close()
#         blog.append(article_dict)
#         print article_dict

# blog profile to json file

In [ ]:
import json
blogjson = json.dumps(blog, ensure_ascii=False)   #bolg為上面list
with open('blog.json', 'w') as a:
    a.write(blogjson.encode('utf-8'))

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL="https://www.pixnet.net/blog/articles/category/29/hot/1"

driver = webdriver.Remote("http://localhost:9515", webdriver.DesiredCapabilities.CHROME)
try:
    driver.get(URL)
#     with open('blog_hot.html', 'w') as f:
#         f.write(driver.page_source.encode('utf-8'))

    for x in range(1,10):
        element = WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.page-next")))
        driver.execute_script("return arguments[0].scrollIntoView()", element)
        time.sleep(5)
        with open('blog_hot/blog_hot{}.html'.format(x), 'w') as f:
            f.write(driver.page_source.encode('utf-8'))
        element.click()
finally:
    driver.close()

In [51]:
import requests
from bs4 import BeautifulSoup
FILE = "blog_hot/blog_hot1.html"

# for page in range(1,25):
with open(FILE, 'r') as f:
    html_file = f.read()
    soup = BeautifulSoup(html_file, 'lxml')
    details0 = soup.select_one('div.featured')
    details = soup.select_one('ol.article-list').select('li')
    
detail_dict1 = {}
detail_list = []
detail_dict['title'] = details0.select_one('a').select_one('img')['alt']
detail_dict['url'] = details0.select_one('a')['href']
detail_dict['author'] = details0.select_one('span.meta').select_one('img')['alt']
detail_dict['date'] = details0.select_one('span.publish').text.split(' ')[1]
detail_dict['reply_count'] = details0.select_one('span.publish').text.split(' ')[5].split('(')[1].split(')')[0]
detail_list.append(detail_dict)
detail_list
# print details0.select_one('a').select_one('img')['alt']
# print details0.select_one('a')['href']
# print details0.select_one('span.meta').select_one('img')['alt']
# print details0.select_one('span.publish').text.split(' ')[1]
# print details0.select_one('span.publish').text.split(' ')[5].split('(')[1].split(')')[0]

for detail in details:
    detail_dict2 = {}
    try:
        if detail.select_one('h3') or detail.select_one('span.meta') != None:          
            detail_dict2['title'] = detail.select_one('h3').select_one('a').text
            detail_dict2['author'] = detail.select_one('span.meta').select_one('a').text
            detail_dict2['url'] = detail.select_one('h3').select_one('a')['href']
            detail_dict2['date'] = detail.select_one('span.meta').text.split(' ')[3]
            detail_dict2['reply_count'] = detail.select_one('span.meta').text.split(' ')[7].split('(')[1].split(')')[0]
            detail_list.append(detail_dict2)
    except Exception as e:
        detail_dict['title'] = ""    

In [52]:
import json
traveljson = json.dumps(detail_list, ensure_ascii=False)
with open('travel.json', 'w') as a:
    a.write(traveljson.encode('utf-8'))

In [55]:
for k in detail_dict2:
    print detail_dict2[k]

http://brian700810.pixnet.net/blog/post/44409382
2016.11.17
0
熱愛旅行的Brian
20161118----新宿好逛景點、百貨列表（按東口、西口、南口劃分）


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL="http://brian700810.pixnet.net/blog/post/44409382"

driver = webdriver.Remote("http://localhost:9515", webdriver.DesiredCapabilities.CHROME)
try:
    driver.get(URL)
#     with open('blog_hot.html', 'w') as f:
#         f.write(driver.page_source.encode('utf-8'))

#     for x in range(1,10):
    element = WebDriverWait(driver,15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.back-to-top")))
    driver.execute_script("return arguments[0].scrollIntoView()", element)
    time.sleep(5)
    with open('blog.html', 'w') as f:
        f.write(driver.page_source.encode('utf-8'))
    element.click()
finally:
    driver.close()

In [3]:
import requests
from bs4 import BeautifulSoup
FILE = "blog.html"
with open(FILE, 'r') as f:
    html_file = f.read()
    soup = BeautifulSoup(html_file, 'lxml')
    articles = soup.select('div.article-content')
    
#     print articles
    
for article in articles:
    japan = article.select('div.article-content-inner')[0].text
    imgs = article.select('div.article-content-inner')[0].select('img')
    for i in imgs:
        print i['src']

https://pic.pimg.tw/brian700810/1479391140-3331773839_n.jpg?v=1479391142
https://pic.pimg.tw/brian700810/1479391226-2267484643_n.jpg?v=1479391229
https://pic.pimg.tw/brian700810/1479391256-813749940_n.jpg
https://pic.pimg.tw/brian700810/1479391294-530520074_n.jpg
https://pic.pimg.tw/brian700810/1479391540-788076669_n.jpg
https://pic.pimg.tw/brian700810/1479391483-1967047473_n.jpg?v=1479391485
https://pic.pimg.tw/brian700810/1479391395-3745793483_n.jpg?v=1479391397
https://pic.pimg.tw/brian700810/1479391345-2613355585_n.jpg
https://pic.pimg.tw/brian700810/1479391603-1343412533_n.jpg
https://pic.pimg.tw/brian700810/1479391657-559577723_n.jpg
https://pic.pimg.tw/brian700810/1479391678-1553540508_n.jpg?v=1479391680
https://pic.pimg.tw/brian700810/1479391801-817174463_n.jpg?v=1479391806
http://panel.pixfs.net/images/icons/tag_blue.gif


In [4]:
print japan


    東京最好逛的地方，很多人都會直奔新宿、涉谷、原宿，喜歡逛街的朋友，不妨花一天的時間火力集中在新宿，選定有興趣的精華店面，享受購物的樂趣。
      
    新宿東口匯集眾多的百貨公司與藥妝店，年輕人不論男女在這裡幾乎可以耗上一整天；
      
    東口再往北走，可抵達不夜城的歌舞伎町，聚集了大量的娛樂場所及餐飲店，也是日本少數的大型紅燈區；
      
    新宿西口多為高樓商辦及飯店林立的商業區，於新宿車站西口可直接通達以中高齡為目標客群的百貨公司，東京都廳也是看夜景的最佳選擇；走在大樓間悠閒地漫步步道，
      
    南口通往都市綠洲「新宿御苑」，年輕人也可以在Lumine. My Load逛街找到喜歡的穿搭款式。
依照車站出口劃分逛街區
    本篇文章依照新宿車站出口分為東口、東口偏北、西口、南口列出大宗推薦逛街景點，供大家行程規劃可集中安排購物景點，每個購物景點可以點選標題進入景點資訊及推薦必逛店家；如果行程比較趕的，可以根據個人喜好挑選喜歡的店家專攻：
東口逛街區
    東口聚集許多百貨公司，並有營業到23:00及24HR的藥妝店，適合逛一整天消磨到半夜也不無聊。
      

1.    丸井百貨（OIOI）：位於新宿的0101共有三家 （本館. Annex, 0101 men, 一館（已歇業））
2.    OIOI men：有許多男士名牌配備0101本館：專為20~30歲女性打造，內部有如大花園，並有各大生活用品及話題品牌進駐，適合選購包包、皮夾、鞋子。

地址：東京都新宿區新宿5-16-4
營業時間：週一～週六11:00～21:00　週日・假日11:00～20:30
交通：JR線新宿車站東口徒步5分鐘

3.    伊勢丹：客群偏中高齡成熟路線，價位偏高， B1是新宿最大土產街。
4.    伊勢丹男館（Isetan Men’s）B1~8F為男性專屬樓層，B1有男用包.男用襪子區，男士內褲，貼身用品區，男鞋等，品牌相當齊全。
5.    LUMINE EST：許多109 超人氣品牌集合的精選購物地帶，並有許多好買的文具店。
6.    H&M：H&M品牌專賣共四層樓，一、二樓的女裝，三樓童裝，地下一樓男裝。衣服飾品皆可在此買到。
7.    藥妝店

    東口有各大連鎖藥妝店SUN DRUG及OS DRU

In [1]:
import jieba
seglist = jieba.cut(japan, cut_all=False)

NameError: name 'japan' is not defined

In [15]:
for word in seglist:
    print word



 
 
 
 
東京
最好
逛
的
地方
，
很多
人
都
會
直奔
新宿
、
涉谷
、
原宿
，
喜歡
逛街
的
朋友
，
不妨
花
一天
的
時間
火力
集中
在
新宿
，
選定
有
興趣
的
精華
店面
，
享受
購物
的
樂趣
。


 
 
 
 
 
 


 
 
 
 
新宿
東口
匯集眾
多
的
百貨
公司
與
藥妝店
，
年
輕人
不論
男女
在
這裡
幾乎
可以
耗上
一整天
；


 
 
 
 
 
 


 
 
 
 
東口
再
往北走
，
可
抵達
不夜城
的
歌舞伎
町
，
聚集
了
大量
的
娛樂場
所
及餐
飲店
，
也
是
日本
少數
的
大型
紅燈區
；


 
 
 
 
 
 


 
 
 
 
新宿
西口
多為
高樓
商辦
及
飯店
林立
的
商業區
，
於
新宿
車站
西口
可
直接
通達
以
中
高齡
為
目標
客群
的
百貨
公司
，
東
京都
廳
也
是
看
夜景
的
最佳
選擇
；
走
在
大樓間
悠閒
地
漫步
步
道
，


 
 
 
 
 
 


 
 
 
 
南口
通往
都市
綠洲
「
新宿
御苑
」
，
年
輕人
也
可以
在
Lumine
.
 
My
 
Load
逛街
找到
喜歡
的
穿
搭
款式
。


依照
車站
出口
劃分
逛街
區


 
 
 
 
本篇
文章
依照
新宿
車站
出口
分為
東口
、
東口
偏北
、
西口
、
南口
列出
大宗
推薦
逛街
景點
，
供
大家
行程
規劃
可
集中
安排
購
物景
點
，
每個
購
物景
點
可以
點選
標題
進入
景點
資訊及
推薦
必逛
店家
；
如果
行程
比較
趕
的
，
可以
根據個
人
喜好
挑選
喜歡
的
店家
專攻
：


東口
逛街
區


 
 
 
 
東口
聚集
許多百貨
公司
，
並有
營業
到
23
:
00
及
24HR
的
藥妝店
，
適合
逛
一整天
消磨
到
半夜
也
不無聊
。


 
 
 
 
 
 




1
.
 
 
 
 
丸井百貨
（
OIOI
）
：
位
於
新宿
的
0101
共有
三家
 
（
本館
.
 
Annex
,
 
0101
 
men
,
 
一館
（
已歇業
）
）


2
.
 
 
 
 
OIOI
 
men
：
有
許多
男士
名牌
配備
0

In [ ]:
from collections import Counter
from operator import itemgetter
res_dict = Counter()
for item in seglist: 
    print item
    if item in res_dict:
        res_dict[item] += 1
    else:
        res_dict[item] = 1

l = []
for key, value in sorted(res_dict.iteritems(), key=itemgetter(1), reverse=True):
    l.append([key.encode('utf-8'), res_dict[key]])

In [18]:
print l

[]
